# meow

In [1]:
import os
# import fitz  # PyMuPDF
import re
import numpy as np
from enum import Enum
# import cv2
# import matplotlib.pyplot as plt
# import pickle
# from collections import Counter, defaultdict

# from PIL import Image, ImageDraw, ImageFont
# import textwrap
# import matplotlib.image as mpimg

# import gzip

class DirNames(Enum):
    TEXT_PICKLE_DIR = 'text_pkl_files'
    PICS_PICKLE_DIR = 'pics_pkl_files'     

class FilePaths(Enum):
    SI_DIR = r'C:\Users\itaro\OneDrive\Documents\GitHub\GatedGCN_NMR\thesis_db\SI_data'
    RAW_DIR = r'C:\Users\itaro\OneDrive\Documents\GitHub\GatedGCN_NMR\thesis_db\raw_data'
    AMENDED_DIR = r'C:\Users\itaro\OneDrive\Documents\GitHub\GatedGCN_NMR\thesis_db\amended_data'
    PPIC_DIR = r'C:\Users\itaro\OneDrive\Documents\GitHub\GatedGCN_NMR\thesis_db\page_pics'
    PICKLE_DIR = r'C:\Users\itaro\OneDrive\Documents\GitHub\GatedGCN_NMR\thesis_db\pickle_files_2911'
    PICKLE_DIR2 = r'C:\Users\itaro\OneDrive\Documents\GitHub\GatedGCN_NMR\thesis_db\pickle_files_2702'
    PICKLE_DIR3 = r'C:\Users\itaro\OneDrive\Documents\GitHub\GatedGCN_NMR\thesis_db\pickle_files_0403'
    PICKLE_DIR4 = r'C:\Users\itaro\OneDrive\Documents\GitHub\GatedGCN_NMR\thesis_db\pickle_files_1003'
    PICKLE_DIR5 = r'C:\Users\itaro\OneDrive\Documents\GitHub\GatedGCN_NMR\thesis_db\pickle_files_1703'
    # RAW_DIR = '/home/yoni4/init_data_18112024/thesis_db/raw_data/'
    # AMENDED_DIR = '/home/yoni4/init_data_18112024/thesis_db/amended_data/'
    # PICKLE_DIR = '/home/yoni4/pickle_files_1603' #'/home/yoni4/init_data_18112024/thesis_db/pickle_files_1003/'
    # PPIC_DIR = '/home/yoni4/page_pics'

raw_pdf_files = [file for file in os.listdir(FilePaths.RAW_DIR.value) if file.endswith('pdf')]
amended_pdf_files = [file for file in os.listdir(FilePaths.AMENDED_DIR.value) if file.endswith('pdf')]
si_pdf_files = [file for file in os.listdir(FilePaths.SI_DIR.value) if file.endswith('pdf')]

In [3]:
from PDF_text_extractor.storeage_obj import load_mol_pic_clusters_dict, load_molecule_segments_dict
from PDF_text_extractor.storeage_obj import ProccessedPdfPictures, ProccessedMoleculeSegments
from PDF_text_extractor.molecule_segment.molecule_segment_obj import MoleculeSegment
from PDF_text_extractor.test_text_line import TestTextLine
from PDF_text_extractor.test_text_sequence import TestTextSequence
from PDF_text_extractor.mol_pic_cluster import MolPicCluster
from PDF_text_extractor.mol_pic import MolPic

pkl_pic_dir = os.path.join(FilePaths.PICKLE_DIR5.value, DirNames.PICS_PICKLE_DIR.value)
pkl_text_dir = os.path.join(FilePaths.PICKLE_DIR5.value, DirNames.TEXT_PICKLE_DIR.value)
     
loaded_pics_dict = load_mol_pic_clusters_dict(pkl_pic_dir)
loaded_molecule_segments_dict = load_molecule_segments_dict(pkl_text_dir)

In [ ]:
def load_default_solvent_dict(solvent_dict=None):
    if solvent_dict is None:
        solvent_dict = {
                ('(CD3)2SO', 'd6-DMSO', 'DMSO', 'DMSO d6', 'd6DMSO', 'dmso'): 'DMSO-d6',
                ('tol-D8', 'd8-toluene', 'toluene-D8'): 'Toluene-d8',
                ('Chloroform-d', 'CHLOROFORM-D', 'cdcl3', 'CDCl3 d1', 'CDCL3'): 'CDCl3', # 'ACETONE-D6' 
                ('MeOD', 'd4-methanol', 'methanol-d4', 'MeOH-d4', 'Methanol-d4'): 'CD3OD',
                ('ACETONE-D6', 'acetone-d6'): 'Acetone-d6', # , 
                ('d8-thf',): 'THF-d8', 
                }
    return solvent_dict

def load_default_nucleui_dict(nucleui_dict=None):
    if nucleui_dict is None:
        nucleui_dict = {
                ('H NMR', '1H-NMR', 'H-NMR', '1HNMR', 'HNMR'): '1H NMR',
                ('2H NMR'): '2D NMR',
                ('{1H} NMR', '}1H} NMR', 'NMR', '29Si NMR'): 'unknown',
                ('19F{1H} NMR'): '19F NMR',
                ('13C-NMR', 'C NMR', '13CNMR', '13C- NMR', '13C{1H} NMR'): '13C NMR', # ,  
                }
    return nucleui_dict

def unify_tokens(given_token, translation_dict):
    unifed_token = given_token
    for token_options, target_token in translation_dict.items():
        if given_token in token_options:
            unifed_token = target_token
    return unifed_token

def unify_tokens_parsed_data(parsed_data, solvent_dict=None, nucleui_dict=None):
    solvent_dict = load_default_solvent_dict(solvent_dict)
    nucleui_dict = load_default_nucleui_dict(nucleui_dict)
    header = parsed_data.get('header')
    header['nucleui'] = unify_tokens(header['nucleui'], nucleui_dict)
    solvent_token = header.get('solvent')
    if solvent_token:
        header['solvent'] = unify_tokens(solvent_token, solvent_dict)
    shifts = parsed_data.get('shifts')
    unified_parsed_data = {'header': header, 'shifts': shifts}
    return unified_parsed_data
    # if type(shifts)==dict:
    #     final_data.append((doc_num, sequence_num, line_num, nmr_text[3], header, shifts))
    # elif len(shifts)==0:
    #     pass
    # elif type(shifts)==list:
    #     final_data.append((doc_num, sequence_num, line_num, nmr_text[3], header, shifts))

In [ ]:
# nucleui - OHC - 7 main options
# frequency - 1 number or just to normalize J?
# solvent - OHC - or pysichal properties ~15
# temp - 1 number
# per shift: - requires padding
#           chem_shift - 1 number - how to deal with range
#           multi_coupling - 2 numbers
#           integration - 1 number
#       nuc  freq   solv  temp   ?sh  4 per shift
# total: 7  +  1   + 12 +   0  +  ?   x  (4) = 24 + 21x4 -> 108

In [ ]:
from PDF_text_extractor.nmr_parser.wrappers import get_nmr_parsed_data_costum

choosen_filename = '000116_Thesis_000115.pdf'
molecule_segments = loaded_molecule_segments_dict.get(choosen_filename) 
mol_pic_clusters = loaded_pics_dict.get(choosen_filename)

def load_default_nucleui_vec_dict(nucleui_vec_dict=None):
    if nucleui_vec_dict is None:
        nucleui_vec_dict = {'1H NMR':  [1, 0, 0, 0, 0, 0, 0],
                            '13C NMR': [0, 1, 0, 0, 0, 0, 0],
                            '19F NMR': [0, 0, 1, 0, 0, 0, 0],
                            '11B NMR': [0, 0, 0, 1, 0, 0, 0],
                            '31P NMR': [0, 0, 0, 0, 1, 0, 0],
                            '2D NMR':  [0, 0, 0, 0, 0, 1, 0],
                            'unk': [0, 0, 0, 0, 0, 0, 1],}
        return nucleui_vec_dict

def vectorize_nucleui(header_dict, nucleui_dict=None):
    nucleui_dict = load_default_nucleui_vec_dict(nucleui_dict)
    nucleui = header_dict.get('nucleui')
    if nucleui in nucleui_dict.keys():
        nucleui_vector = nucleui_dict.get(nucleui)
    else:
        nucleui_vector = nucleui_dict.get('unk')
    return nucleui_vector

def frequency_str_to_num(frequency_str):
    return float(frequency_str.rstrip().rstrip('MHz').rstrip().rstrip(','))

def get_default_frequency(nucleui):
    if nucleui == '1H NMR':
        frequency = 400
    elif nucleui == '13C NMR':
        frequency = 116
    else:
        frequency = -1
    return frequency    

def vectorize_frequency(header_dict):
    frequency_str = header_dict.get('frequency')                
    if frequency_str:
        frequency_vector = [frequency_str_to_num(frequency_str)]
    else:
        nucleui = header_dict.get('nucleui')
        frequency_vector = [get_default_frequency(nucleui)]
    return frequency_vector

def load_default_solvent_vec_dict(solvent_vec_dict=None):
    if solvent_vec_dict is None:
        solvent_vec_dict = {'CDCl3':      [1,0,0,0,0,0,0,0,0,0,0,0],
                            'DMSO-d6':    [0,1,0,0,0,0,0,0,0,0,0,0],
                            'CD3OD':      [0,0,1,0,0,0,0,0,0,0,0,0],
                            'CD2Cl2':     [0,0,0,1,0,0,0,0,0,0,0,0],
                            'C6D6':       [0,0,0,0,1,0,0,0,0,0,0,0],
                            'THF-d8':     [0,0,0,0,0,1,0,0,0,0,0,0],
                            'D2O':        [0,0,0,0,0,0,1,0,0,0,0,0],
                            'Acetone-d6': [0,0,0,0,0,0,0,1,0,0,0,0],
                            'Toluene-d8': [0,0,0,0,0,0,0,0,1,0,0,0],
                            'DMF-d7':     [0,0,0,0,0,0,0,0,0,1,0,0],
                            'CD3CN':      [0,0,0,0,0,0,0,0,0,0,1,0],
                            'unk':       [0,0,0,0,0,0,0,0,0,0,0,1],
                            'CDCl3 over K2CO3': [1,0,0,0,0,0,0,0,0,0,0,1],
                            'd8-toluene/CD3CN mixture': [0,0,0,0,0,0,0,0,1,0,1,0],
                            }
        return solvent_vec_dict

def vectorize_solvent(header_dict, solvent_dict=None):
    solvent_dict = load_default_solvent_vec_dict(solvent_dict)
    solvent = header_dict.get('solvent')
    if solvent in solvent_dict.keys():
        solvent_vector = solvent_dict.get(solvent)
    else:
        solvent_vector = solvent_dict.get('unk')
    return solvent_vector

def get_header_vector(header_dict, nucleui_dict=None, solvent_dict=None):
    nucleui_vector = vectorize_nucleui(header_dict, nucleui_dict)
    frequency_vector = vectorize_frequency(header_dict)
    solvent_vector = vectorize_solvent(header_dict, solvent_dict)
    header_vector = nucleui_vector + frequency_vector + solvent_vector
    return header_vector

def load_default_multiplicity_tr_dict(multiplicity_tr_dict=None):
    if multiplicity_tr_dict is None:
        multiplicity_tr_dict = {'s': 1,
                            'd': 2,
                            't': 3,
                            'q': 4,
                            'u': 5,
                            'x': 6,
                            'p': 7,
                            'o': 8,
                            'n': 9,
                            'm': 10,
                            'b': 11,
                            'unk': -1}
    return multiplicity_tr_dict

def transform_multiplicity(multiplicity):
    edited_multiplicity = multiplicity.strip().rstrip()
    edited_multiplicity = edited_multiplicity.replace('quintet', 'u').replace('quin', 'u')
    edited_multiplicity = edited_multiplicity.replace('(br)', 'b').replace('br', 'b')
    edited_multiplicity = edited_multiplicity.replace('septet', 'p').replace('sept', 'p')
    edited_multiplicity = edited_multiplicity.replace('octet', 'o')
    edited_multiplicity = edited_multiplicity.replace('sxt', 'x')
    edited_multiplicity = edited_multiplicity.replace('t with further splitting', 't')
    return edited_multiplicity

def vectorize_multiplicity(tr_multiplicity, final_len, multiplicity_dict=None):
    multiplicity_dict = load_default_multiplicity_tr_dict(multiplicity_dict)
    multiplicity_vector = []
    for multi_letter in tr_multiplicity:
        multi_num = multiplicity_dict.get(multi_letter)
        if multi_num:
            multiplicity_vector.append(multi_num)
        else:
            multiplicity_vector.append(multiplicity_dict.get('unk'))
    while len(multiplicity_vector)<final_len:
        multiplicity_vector.append(multiplicity_dict.get('unk'))
    return multiplicity_vector

def transform_coupling(coupling):
    edited_coupling_str = coupling.strip(' ').strip('J').strip(' ').strip('=').strip(' ').rstrip('Hz').rstrip(' ').rstrip('.')
    coupling_list = edited_coupling_str.split(',')
    return coupling_list

def deal_with_num_str(num_str, splitter=' '):
    final_num = [-999.0]
    try:
        final_num = [float(num_str)]
    except:
        try:
            final_num = [-1*float(num_str[1:])]
        except:
            if num_str and len(num_str)<=5:
                final_num = [float(num_str)]
            elif num_str:
                num_of_dpoint = num_str.count('.')
                if num_of_dpoint<=1:
                    final_num = [float(num_str)]
                else:
                    split_result = num_str.split(splitter)
                    final_num = [float(inner_str.rstrip('.')) for inner_str in split_result]
    return final_num

def vectorize_coupling(tr_coupling, final_len, frequency=1000):
    coupling_vector = []
    for inner_coupling in tr_coupling:
        edited_inner_coupling = inner_coupling.strip(' ').rstrip('|').rstrip(' ')
        coupling_vector.extend(deal_with_num_str(edited_inner_coupling))
    norm_coupling_vector = [round(100*c/frequency,2) for c in coupling_vector]
    while len(norm_coupling_vector)<final_len:
        norm_coupling_vector.append(-0.5)
    return norm_coupling_vector

# def get_shifts_vector(shift_dict_list, nucleui, frequency, per_shift_feat=4, max_peaks=24):
#     total_shift_vector = []
#     if len(shift_dict_list)>0 and type(shift_dict_list)==list:
#         num_of_curr_shift = 0
#         total_shift_vector = []
#         for shift_dict in shift_dict_list:
#             num_of_curr_shift+=1
#             chemical_shift = np.mean(deal_with_num_str(shift_dict.get('shift'), splitter='-'))
#             multiplicity_vector = vectorize_multiplicity([], final_len=per_shift_feat)
#             coupling_vector = vectorize_coupling([], final_len=per_shift_feat, frequency=frequency)                       
#             if nucleui == '1H NMR':
#                 multiplicity = shift_dict.get('multiplicity')
#                 if multiplicity:
#                     transformed_multiplicity = transform_multiplicity(multiplicity)
#                     multiplicity_vector = vectorize_multiplicity(transformed_multiplicity, final_len=per_shift_feat)
#                     coupling = shift_dict.get('coupling')
#                     if coupling:
#                         transformed_coupling = transform_coupling(coupling)
#                         coupling_vector = vectorize_coupling(transformed_coupling, final_len=per_shift_feat, frequency=frequency)
#             shift_vector = [chemical_shift]
#             for mult, coup in zip(multiplicity_vector, coupling_vector):
#                 shift_vector+=[mult, coup]
#             total_shift_vector+=shift_vector
#     while num_of_curr_shift<max_peaks:
#         chemical_shift = 0
#         multiplicity_vector = vectorize_multiplicity([], final_len=per_shift_feat)
#         coupling_vector = vectorize_coupling([], final_len=per_shift_feat, frequency=frequency)
#         shift_vector = [chemical_shift]
#         for mult, coup in zip(multiplicity_vector, coupling_vector):
#             shift_vector+=[mult, coup]
#         num_of_curr_shift+=1
#         total_shift_vector+=shift_vector
#     return total_shift_vector

def compute_chemical_shift(shift_dict):
    """Extract and compute the mean chemical shift from a shift dictionary."""
    raw_shift = shift_dict.get('shift')
    return np.mean(deal_with_num_str(raw_shift, splitter='-'))

def get_multiplicity_vector(shift_dict, per_shift_feat):
    """Return the multiplicity vector from the shift dictionary."""
    multiplicity = shift_dict.get('multiplicity')
    if multiplicity:
        transformed = transform_multiplicity(multiplicity)
        return vectorize_multiplicity(transformed, final_len=per_shift_feat)
    return vectorize_multiplicity([], final_len=per_shift_feat)

def get_coupling_vector(shift_dict, per_shift_feat, frequency):
    """Return the coupling vector from the shift dictionary."""
    coupling = shift_dict.get('coupling')
    if coupling:
        transformed = transform_coupling(coupling)
        return vectorize_coupling(transformed, final_len=per_shift_feat, frequency=frequency)
    return vectorize_coupling([], final_len=per_shift_feat, frequency=frequency)

def create_shift_vector(chemical_shift, multiplicity_vector, coupling_vector):
    """Combine chemical shift, multiplicity, and coupling vectors into one flat vector."""
    shift_vector = [chemical_shift]
    for mult, coup in zip(multiplicity_vector, coupling_vector):
        shift_vector.extend([mult, coup])
    return shift_vector

def pad_shift_vectors(total_shift_vector, current_count, max_peaks, per_shift_feat, frequency):
    """Pad with zeroed shift vectors until max_peaks is reached."""
    while current_count < max_peaks:
        zero_shift = 0
        multiplicity_vector = vectorize_multiplicity([], final_len=per_shift_feat)
        coupling_vector = vectorize_coupling([], final_len=per_shift_feat, frequency=frequency)
        shift_vector = create_shift_vector(zero_shift, multiplicity_vector, coupling_vector)
        total_shift_vector.extend(shift_vector)
        current_count += 1
    return total_shift_vector

def get_shifts_vector(shift_dict_list, nucleui, frequency, per_shift_feat=4, max_peaks=24):
    """Generate the final shift vector representation for all peaks."""
    total_shift_vector = []
    if not isinstance(shift_dict_list, list) or not shift_dict_list:
        return pad_shift_vectors(total_shift_vector, 0, max_peaks, per_shift_feat, frequency)

    for i, shift_dict in enumerate(shift_dict_list):
        chemical_shift = compute_chemical_shift(shift_dict)
        if nucleui == '1H NMR':
            multiplicity_vector = get_multiplicity_vector(shift_dict, per_shift_feat)
            coupling_vector = get_coupling_vector(shift_dict, per_shift_feat, frequency)
        else:
            multiplicity_vector = vectorize_multiplicity([], final_len=per_shift_feat)
            coupling_vector = vectorize_coupling([], final_len=per_shift_feat, frequency=frequency)
        shift_vector = create_shift_vector(chemical_shift, multiplicity_vector, coupling_vector)
        total_shift_vector.extend(shift_vector)

    return pad_shift_vectors(total_shift_vector, len(shift_dict_list), max_peaks, per_shift_feat, frequency)

def vectorize_parsed_data(parsed_data, max_peaks=24, nucleui_dict=None, solvent_dict=None):
    header = parsed_data.get('header')
    header_vector = get_header_vector(header, nucleui_dict=nucleui_dict, solvent_dict=solvent_dict)
    nucleui = header.get('nucleui')
    frequency = header_vector[7]
    if frequency == 0:
        frequency = get_default_frequency(nucleui)
    shifts = parsed_data.get('shifts')
    PER_SHIFT_FEAT = 4
    total_shift_vector = get_shifts_vector(shift_dict_list=shifts, nucleui=nucleui, frequency=frequency, per_shift_feat=PER_SHIFT_FEAT, max_peaks=max_peaks)
    final_vector = header_vector + total_shift_vector
    return final_vector

for molecule_segment in molecule_segments:
    if molecule_segment.nmr_text_line_list:
        for nmr_line in molecule_segment.nmr_text_line_list:
            parsed_data_costum = get_nmr_parsed_data_costum(nmr_line)
            if parsed_data_costum.get('shifts'):
                unified_parsed_data = unify_tokens_parsed_data(parsed_data_costum)
                text_vector = vectorize_parsed_data(unified_parsed_data)
                print(text_vector)      

[1, 0, 0, 0, 0, 0, 0, 400, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1.8, -1, -0.5, -1, -0.5, -1, -0.5, -1, -0.5, 1.9, -1, -0.5, -1, -0.5, -1, -0.5, -1, -0.5, 1.1, 3, -0.5, 3, -0.5, -1, -0.5, -1, -0.5, 0, -1, -0.5, -1, -0.5, -1, -0.5, -1, -0.5, 0, -1, -0.5, -1, -0.5, -1, -0.5, -1, -0.5, 0, -1, -0.5, -1, -0.5, -1, -0.5, -1, -0.5, 0, -1, -0.5, -1, -0.5, -1, -0.5, -1, -0.5, 0, -1, -0.5, -1, -0.5, -1, -0.5, -1, -0.5, 0, -1, -0.5, -1, -0.5, -1, -0.5, -1, -0.5, 0, -1, -0.5, -1, -0.5, -1, -0.5, -1, -0.5, 0, -1, -0.5, -1, -0.5, -1, -0.5, -1, -0.5, 0, -1, -0.5, -1, -0.5, -1, -0.5, -1, -0.5, 0, -1, -0.5, -1, -0.5, -1, -0.5, -1, -0.5, 0, -1, -0.5, -1, -0.5, -1, -0.5, -1, -0.5, 0, -1, -0.5, -1, -0.5, -1, -0.5, -1, -0.5, 0, -1, -0.5, -1, -0.5, -1, -0.5, -1, -0.5, 0, -1, -0.5, -1, -0.5, -1, -0.5, -1, -0.5, 0, -1, -0.5, -1, -0.5, -1, -0.5, -1, -0.5, 0, -1, -0.5, -1, -0.5, -1, -0.5, -1, -0.5, 0, -1, -0.5, -1, -0.5, -1, -0.5, -1, -0.5, 0, -1, -0.5, -1, -0.5, -1, -0.5, -1, -0.5, 0, -1, -0.5, -1, -0.5, -1, -0.